Note: To display the density grid calculations, in VisIt, load in the .cube file (must select file type and deselect default option), and deselect options to apply options to all plots/windows. When making density grid plots, use the minimum and maximum energy values displayed inthe legend (don't turn off annotations in VisIt immediately) to *normalize* the color gradient. Make a low quality plot and find a good viewing angle; then, increase the resolution and print screen as png. 

TODO: try making density grid plots using alternative software posted in slack.

In [1]:
using PorousMaterials
using JLD2

┌ Info: Precompiling PorousMaterials [68953c7c-a3c7-538e-83d3-73516288599e]
└ @ Base loading.jl:1317
┌ Warning: Package Xtals does not have LightGraphs in its dependencies:
│ - If you have Xtals checked out for development and have
│   added LightGraphs as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with Xtals
└ Loading LightGraphs into Xtals from project dependency, future warnings for Xtals are suppressed.
┌ Warning: Package MetaGraphs does not have LightGraphs in its dependencies:
│ - If you have MetaGraphs checked out for development and have
│   added LightGraphs as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with MetaGraphs
└ Loading LightGraphs into MetaGraphs from project dependency, future warnings for MetaGraphs are suppressed.
┌ Warning: The call to compilecache failed to cr

In [2]:
set_paths(joinpath(pwd(), "../../data"))
# @eval PorousMaterials PATH_TO_CRYSTALS=joinpath(pwd(),"structural_relaxation","post-relaxation_cifs")
# @eval PorousMaterials PATH_TO_CRYSTALS=joinpath(pwd(), "images/grid_images/replicated_xtals/")

In [3]:
###
#  Set flag for desired calculation
###
calc_energy_grid   = false # flag for energy grid calculations
write_density_grid = false  # flag for extracting density grid from *.jld2 sim files 

false

In [4]:
# preset dictionary vlues
params = Dict(:adsorbate   => "Xe", #Kr
              :ljff        => "UFF",
              :grid_points => (100, 100, 100), 
              :repfactors  => (1, 1, 1),
              :temperature => 298.0, 
              :n_rotations => 750,
              :energy_min  => Tuple{Float64, CartesianIndex{3}},
              :xyz_file    => false,
              :vtk_file    => false)

Dict{Symbol, Any} with 9 entries:
  :ljff        => "UFF"
  :adsorbate   => "Xe"
  :grid_points => (100, 100, 100)
  :temperature => 298.0
  :energy_min  => Tuple{Float64, CartesianIndex{3}}
  :xyz_file    => false
  :vtk_file    => false
  :repfactors  => (1, 1, 1)
  :n_rotations => 750

In [5]:
# I need to put place the adsorbate at the location of the minimum energy
function adsorbate_in_mof(energy_grid_params::Dict{String,Dict{Symbol,Any}}, xtal::Crystal, grid::Grid)
    filename = energy_grid_params[xtal.name][:adsorbate] * "_in_" * xtal.name
    
    molecule = Molecule(energy_grid_params[xtal.name][:adsorbate])
    xyz = Tuple([energy_grid_params[xtal.name][:energy_min][2][i] for i in 1:3])
    
    xf_minE = id_to_xf(xyz, grid.n_pts) # fractional coords of min
    x_minE  = Cart(Frac(xf_minE), xtal.box) # Cartesian coords of min
    # make struct
    xe_atom_c = Atoms(1, [molecule.species], x_minE) 

    return write_xyz(xe_atom_c, filename)
end

adsorbate_in_mof (generic function with 1 method)

In [6]:
xtal_names = ["NiPyC2_experiment.cif", 
              "Pn_Ni-PyC-NH2_sc222.cif",
              "NiPyC2_relax_sc211_meta_functionalized_NH2_pbesol_relax.cif",
              "NiPyC2_relax_sc211_meta_functionalized_CH3_pbesol_relax.cif",
              "NiPyC2_relax_sc211_meta_functionalized_N-C-O_pbesol_relax.cif"]

energy_grid_params = Dict(zip(xtal_names, [params for i in 1:length(xtal_names)]))

Dict{String, Dict{Symbol, Any}} with 1 entry:
  "Pn_Ni-PyC-NH2_sc222.cif" => Dict(:ljff=>"UFF", :adsorbate=>"Xe", :grid_point…

In [7]:
# special case
energy_grid_params["NiPyC2_experiment.cif"][:repfactors] = (2, 2, 2)

LoadError: KeyError: key "NiPyC2_experiment.cif" not found

## Density Grid from `.jld2` File

In [8]:
if write_density_grid
    for xtal_name in xtal_names
        # load crystal and replicate 
        crystal = Crystal(xtal_name)

        # assign energy calculation parameters
        mol    = Molecule(energy_grid_params[xtal_name][:adsorbate])
        ljff   = LJForceField(energy_grid_params[xtal_name][:ljff])
        temp   = energy_grid_params[xtal_name][:temperature]      
        pressures = [0.01, 0.1, 1.0]
        nburn     = 50000
        nsample   = 50000

        # loop over pressures
        for p in pressures
            filename = μVT_output_filename(crystal, [mol], temp, [p], ljff, nburn, nsample)
            println(filename)

            @load joinpath(PorousMaterials.rc[:paths][:simulations], filename) results

            grid = results["density grid"]

            # find the value and location of the minimum energy (useful when plotting) 
            energy_grid_params[xtal_name][:energy_min] = findmin(grid.data)
            println(xtal_name, " energy min = ", energy_grid_params[xtal_name][:energy_min])

            # write cube file using crystal name for filename
            write_cube(grid, filename)
        end

        # write xyz file
        if energy_grid_params[xtal_name][:xyz_file]
            atoms_c = Cart(crystal.atoms, crystal.box)
            write_xyz(crystal)
        end
        # write vtk file
        if energy_grid_params[xtal_name][:vtk_file]
            write_vtk(crystal.box, crystal.name)
        end
    end
end

## Energy Grid Calculations

In [14]:
if calc_energy_grid
    for xtal_name in xtal_names
        # load crystal and replicat (if needed)
        crystal = replicate(Crystal(xtal_name), energy_grid_params[xtal_name][:repfactors])
        strip_numbers_from_atom_labels!(crystal)

        # assign energy calculation parameters
        mol    = Molecule(energy_grid_params[xtal_name][:adsorbate])
        ljff   = LJForceField(energy_grid_params[xtal_name][:ljff])
        points = energy_grid_params[xtal_name][:grid_points]
        temp   = energy_grid_params[xtal_name][:temperature]
        rot    = energy_grid_params[xtal_name][:n_rotations]

        # perform energy grid calculation
        grid = energy_grid(crystal, mol, ljff; resolution=points, temperature=temp, n_rotations=rot)

        # find the value and location of the minimum energy 
        energy_grid_params[xtal_name][:energy_min] = findmin(grid.data)
        println(xtal_name, " energy min = ", energy_grid_params[xtal_name][:energy_min])

        # write cube file using crystal name for filename
        write_cube(grid, "Egrid_" * crystal.name)

        # write xyz file
        if energy_grid_params[xtal_name][:xyz_file]
            atoms_c = Cart(crystal.atoms, crystal.box)
            write_xyz(crystal)
        end

        # write vtk file
        if energy_grid_params[xtal_name][:vtk_file]
            write_vtk(crystal)
        end

    #     adsorbate_in_mof(energy_grid_params, crystal, grid)
    end
end

Computing energy grid of Xe in Pn_Ni-PyC-NH2_sc222.cif
	Regular grid (in fractional space) of 100 by 100 by 100 points superimposed over the unit cell.
Pn_Ni-PyC-NH2_sc222.cif energy min = (-31.79297784086987, CartesianIndex(76, 90, 94))
	See /home/ng/DTRA/visualizations/grid_images/../../data/grids/Egrid_Pn_Ni-PyC-NH2_sc222.cif.cube
